In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns


from matplotlib import pyplot as plt

In [2]:
program = "maximal_matching"
program = "dijkstra_token_ring"
graph_name = "implicit_graph_n7"

no_simulations = 10000
fault_interval = 1
sched = 0

selected_nodes = [0, 1, 5, 6]

merge_files = [
    f"w_exp_{program}__{graph_name}__{sched}__{no_simulations}__{fault_interval}__{n}__duong"
    for n in selected_nodes
]

In [3]:
dfs = [pd.read_csv(f"{os.path.join("results", program, f)}.csv").drop('Count', axis=1) for f in merge_files]
for df in dfs:
    df['Steps'] = np.ceil(df['Steps']).astype(int)

In [4]:
dfs[2]

,Steps,exp_fit
0,13,156.582883
1,14,153.393007
2,15,150.268160
3,16,147.207018
4,17,144.208282
...,...,...
452,453,0.126670
453,454,0.126381
454,455,0.126098
455,456,0.125821


In [5]:
dfs[0]

,Steps,exp_fit
0,11,165.442381
1,12,162.007854
2,13,158.644701
3,14,155.351436
4,15,152.126610
...,...,...
394,395,0.211982
395,396,0.211106
396,397,0.210248
397,398,0.209408


In [6]:
dfs[1]

,Steps,exp_fit
0,12,157.361582
1,13,154.191949
2,14,151.086254
3,15,148.043210
4,16,145.061551
...,...,...
433,434,0.257980
434,435,0.257513
435,436,0.257056
436,437,0.256608


In [7]:
df_merged = dfs[0].copy()
for i in range(1, len(dfs)):
    df_merged = pd.merge(
        df_merged,
        dfs[i],
        on=["Steps"],
        suffixes=(f"__{selected_nodes[i - 1]}", ""),
    ).fillna(0)

df_merged.rename(columns={df_merged.columns[-1]: f'exp_fit__{selected_nodes[-1]}'}, inplace=True)
df_merged = df_merged.set_index('Steps')
df_merged

,exp_fit__0,exp_fit__1,exp_fit__5,exp_fit__6
Steps,,,,
13,158.644701,154.191949,156.582883,169.735219
14,155.351436,151.086254,153.393007,165.942144
15,152.126610,148.043210,150.268160,162.233812
16,148.968798,145.061551,147.207018,158.608330
17,145.876609,142.140039,144.208282,155.063846
...,...,...,...,...
395,0.211982,0.287103,0.160254,-0.013217
396,0.211106,0.286049,0.159281,-0.013864
397,0.210248,0.285017,0.158327,-0.014496


In [8]:
for col in df_merged.columns:
    df_merged.rename(columns={col: f"Exponential fit of node {col.split('__')[1]}"}, inplace=True)

df_merged

,Exponential fit of node 0,Exponential fit of node 1,Exponential fit of node 5,Exponential fit of node 6
Steps,,,,
13,158.644701,154.191949,156.582883,169.735219
14,155.351436,151.086254,153.393007,165.942144
15,152.126610,148.043210,150.268160,162.233812
16,148.968798,145.061551,147.207018,158.608330
17,145.876609,142.140039,144.208282,155.063846
...,...,...,...,...
395,0.211982,0.287103,0.160254,-0.013217
396,0.211106,0.286049,0.159281,-0.013864
397,0.210248,0.285017,0.158327,-0.014496


In [9]:
fontsize = 20

def get_title():
    return f"Simulation curve fit - {program} | {graph_name} | Sched: {sched} | N: {no_simulations:,} | FI: {fault_interval}"


In [10]:
# sns.lineplot(df_merged, dashes={f"Exponential fit of node {i}":"" for i in selected_nodes}, linewidth=1)

In [11]:
plt.figure(figsize=(16, 8))
ax = sns.lineplot(data=df_merged, dashes={f"Exponential fit of node {i}":"" for i in selected_nodes}, linewidth=1)
ax.set_title(get_title(), fontdict={"fontsize": fontsize})

ax.tick_params(axis="x", labelsize=fontsize)
ax.tick_params(axis="y", labelsize=fontsize)
ax.xaxis.label.set_size(fontsize)
ax.yaxis.label.set_size(fontsize)


ax.set_xlabel("Steps")
ax.set_ylabel("Count")

plt.rc("font", size=fontsize)
plt.legend(fontsize=fontsize * 0.9)

plt.savefig(
    f"exp_curve_fit_{program}__{graph_name}__{''.join([str(i) for i in selected_nodes])}.png",
    bbox_inches="tight",
)
plt.close()